___
## Módulo 5: Tratamiento de series temporales.
___

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

### 5.1. Tratamiento de series temporales I

**5.1.1.** Genera un vector de fechas con 15 periodos de frecuencia mensual que empiece el 2015-1-1.

In [ ]:
dates = pd.date_range(start = '2015-1-1', periods=15, freq='MS')
dates

**5.1.2.** A continuación, genera 4 columnas de números aleatorios - que sigan una distribución normal estándar centrada en 10 y con una desviación típica de 2 - de la misma longitud que el vector de fechas. Redondea los números aleatorios a 6 decimales. 

In [ ]:
dates.shape

In [ ]:
n = dates.shape[0]
sigma = 2
mu = 10
cols = sigma * np.random.randn(n, 4) + mu
cols = np.round(cols, decimals=6)
cols

**5.1.3.** Genera a partir de ambos inputs un DataFrame que tenga el vector de fechas como índice.

In [ ]:
data = pd.DataFrame(cols, index=dates)
data

**5.1.4.** Accede a todos los elementos del año 2015, a todos los datos de abril del mismo año, y a los datos que se encuentran entre abril y septiembre del mismo año.

In [ ]:
data['2015']

In [ ]:
data['2015-04']

In [ ]:
data['2015-04':'2015-09']

**5.1.5.** Para calcular la tasa de variación se puede usar 'pct_change'. Usa esta función para calcular la tasa de variación mensual y trimestral.

In [ ]:
data.resample('M').first().pct_change()

In [ ]:
data.resample('Q').first().pct_change()

___
### 5.2. Tratamiento de series temporales II

**5.2.1.** Carga los csvs de datos ibex_div, ibex, NTGY, REE, SAN, pon la fecha como índice.

In [ ]:
san = pd.read_csv('SAN.csv', parse_dates=True, index_col=0)
ree = pd.read_csv('REE.csv', parse_dates=True, index_col=0)
ntgy = pd.read_csv('NTGY.csv', parse_dates=True, index_col=0)
ibex = pd.read_csv('ibex.csv', parse_dates=True, index_col=0)
ibex_div = pd.read_csv('ibex_div.csv', parse_dates=True, index_col=0)

**5.2.2.**  Calcula el retorno anualizado del Ibex con dividendos y del Ibex.

In [ ]:
years = (ibex.index[-1]-ibex.index[0]).days / 365
total_return = ibex.close[-1] / ibex.close[0] - 1
total_return

In [ ]:
year_return_ibex = (total_return+1)**(1/years) -1
year_return_ibex

In [ ]:
years = (ibex_div.index[-1]-ibex_div.index[0]).days / 365
total_return = ibex_div.close[-1] / ibex_div.close[0] - 1 
total_return

In [ ]:
return_ibex_div = (1 + total_return)**(1/years) - 1
return_ibex_div

In [ ]:
ibex_div.close.plot()
ibex.close.plot()

**5.2.3.**  Calcula la serie de retornos anuales  del Ibex con dividendos y del Ibex.

In [ ]:
first_day_year_value_ibex = ibex.close.resample('YS').first()
first_day_year_value_ibex

In [ ]:
first_day_year_value_ibex_div = ibex_div.close.resample('YS').first()

In [ ]:
returns_year_ibex = first_day_year_value_ibex.pct_change().dropna()
returns_year_ibex_div = first_day_year_value_ibex_div.pct_change().dropna()

**5.2.4.**  Realiza un gráfico de barras comparándolos.

In [ ]:
returns_year_ibex.index = returns_year_ibex.index.year 
returns_year_ibex_div.index = returns_year_ibex_div.index.year 

In [ ]:
data = pd.DataFrame({
    'ibex': returns_year_ibex,
    'ibex div': returns_year_ibex_div
})

In [ ]:
data.plot.bar()

**5.2.5.**  Compara los retornos anuales del Ibex con los de SAN.

In [ ]:
first_day_year_value_san = san.close.resample('YS').first()
returns_year_san = first_day_year_value_san.pct_change().dropna()
returns_year_san.index = returns_year_san.index.year 

In [ ]:
df = pd.DataFrame({
    'san': returns_year_san,
    'ibex': returns_year_ibex,
})
df.plot.bar()

**5.2.6.**  Calcula la correlación del SAN, REE y NTGY con el IBEX con dividendos.

In [ ]:
data = pd.DataFrame({
    'san': san.close,
    'ree': ree.close,
    'ntgy': ntgy.close,
    'ibex': ibex.close,
    'ibex_div': ibex_div.close,
})

In [ ]:
data = data.dropna()

In [ ]:
returns_data = np.log(data).diff()

In [ ]:
returns_data.corr()

**5.2.7.** Ahora calcula la correlación rolada de 100 días para los activos del ejercico anterior.

In [ ]:
rolling_corr = returns_data.iloc[:, 2:].rolling(100).corr(returns_data['ibex_div'])

In [ ]:
rolling_corr.plot(figsize=(20,10))

**5.2.8.** Calcula la media movil de 30 y 200 dias de REE y realiza una figura junto con la serie de precios originales

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
ree.close.rolling(30).mean().plot(ax=ax, label='M30', color='k')
ree.close.rolling(200).mean().plot(ax=ax, label='M200')
ree.close.plot(ax=ax, label='precio')
ax.legend()

**5.2.9.**  Usando el precio de cierre del Ibex, calcula las velas mensuales y anuales.

In [ ]:
ibex_month_candle = ibex.close.resample('MS').ohlc()
ibex_month_candle.head()

In [ ]:
ibex_year_candle = ibex.close.resample('YS').ohlc()
ibex_year_candle.head()

**5.2.10.** Píntalas utilizando un gráfico de barras.

In [ ]:
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates

def plot_candle(df, width=.5, figsize=(10, 7), tick_formater='%Y-%m'):        
    m_data = df[['open', 'high', 'low', 'close']].values
    days_m_dates_format = mdates.date2num(df.index.values) 
    data_plot = np.column_stack((days_m_dates_format, m_data))
        
    fig, ax = plt.subplots(figsize=figsize)
    _ = candlestick_ohlc(ax,
                         data_plot,
                         width=width,
                         colorup='green',
                         colordown='red')
    _ = ax.xaxis.set_major_formatter(mdates.DateFormatter(tick_formater))
    return fig, ax

In [ ]:
plot_candle(ibex_month_candle, width=10)

In [ ]:
plot_candle(ibex_year_candle, width=100)

**5.2.11.**  Calcula la beta de los 3 activos NTGY, REE y SAN, Recuerda:

$\beta = \frac{cov(R_m, R_s)}{var(R_m)}$

Donde $R_m$ y $R_s$ son la serie de retornos del índice y de la acción.

In [ ]:
data = pd.DataFrame({
    'san': san.close,
    'ree': ree.close,
    'ntgy': ntgy.close,
    'ibex': ibex.close,
    'ibex_div': ibex_div.close,
})
data = data.dropna()

In [ ]:
# usamos retornos log
returns = np.log(data).diff()
#returns = data.pct_change()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(returns.ibex, returns.san, 'o',)
ax.set_xlabel('$R_{ibex}$')
ax.set_ylabel('$R_{SAN}$')

In [ ]:
cov_mat = returns.cov()

In [ ]:
cov_mat

In [ ]:
cov_mat['ibex'].iloc[:3] / returns.ibex.var()

Para los últimos 3 años:

In [ ]:
cov_mat = returns['2017':].cov()

In [ ]:
cov_mat['ibex'].iloc[:3] / returns['2017':].ibex.var()

Comparar con investing:
 - SAN https://www.investing.com/equities/banco-santander
 - REE https://www.investing.com/equities/red-electrica
 - NTGY https://www.investing.com/equities/gas-natural-sdg

**5.2.12.** Ahora calcula la beta rolada de 100 días para los activos del ejercico **5.2.11.**

In [ ]:
cov_r_s_r_ibex = returns.loc[:, 'san'].rolling(100).cov(returns['ibex_div']) 
var_r_ibex = returns.loc[:, 'ibex_div'].rolling(100).cov(returns['ibex_div']) 

In [ ]:
rolling_beta = cov_r_s_r_ibex / var_r_ibex

In [ ]:
rolling_beta.plot()